# The CNS North Korea Missile Test Database

In [3]:
%load_ext lab_black

In [4]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_grid as altgrid
import numpy as np
import us
import requests

In [5]:
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [6]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Download latest version of the database from [here](https://beyondparallel.csis.org/database-north-korean-provocations/)

In [7]:
url = "https://www.nti.org/wp-content/uploads/2021/10/north_korea_missile_test_database_current-for-website.xlsx"

In [8]:
resp = requests.get(url)
output = open("data/raw/north_korea_missile_test_database.xls", "wb")
output.write(resp.content)
output.close()

#### Read data

In [21]:
src = pd.read_excel(
    "data/raw/north_korea_missile_test_database.xls", skiprows=1, parse_dates=["Date"]
).sort_values("Date", ascending=False)[
    [
        "Date",
        "Date Entered/Updated",
        "Launch Time (UTC)",
        "Missile Name",
        "Missile Type",
        "Facility Name",
        "Facility Location",
        "Facility Latitude",
        "Facility Longitude",
        "Landing Location",
        "Apogee",
        "Distance Travelled",
        "Confirmation Status",
        "Test Outcome",
        "Additional Information",
        "Source(s)",
    ]
]

In [22]:
src.columns

Index(['Date', 'Date Entered/Updated', 'Launch Time (UTC)', 'Missile Name',
       'Missile Type', 'Facility Name', 'Facility Location',
       'Facility Latitude', 'Facility Longitude', 'Landing Location', 'Apogee',
       'Distance Travelled', 'Confirmation Status', 'Test Outcome',
       'Additional Information', 'Source(s)'],
      dtype='object')

In [23]:
src.columns = (
    src.columns.str.lower()
    .str.replace(" ", "_", regex=False)
    .str.replace("/", "_", regex=False)
)

#### Dates

In [92]:
src["year"] = src["date"].dt.year.astype(str)
src["month"] = src["date"].dt.month
src["day"] = src["date"].dt.day
src["day"] = src["date"].dt.day_name()

In [93]:
df = src.copy()

In [94]:
df.head()

,date,date_entered_updated,launch_time_(utc),missile_name,missile_type,facility_name,facility_location,facility_latitude,facility_longitude,landing_location,apogee,distance_travelled,confirmation_status,test_outcome,additional_information,source(s),year,month,day
202,2022-10-08,2022-10-10,16:53:00,KN-25,SRBM,Munchon,"Munchon, Kangwong Province",39.2815,127.377,Sea of Japan or East Sea,90 km,350 km,Confirmed,Success,2 of 2 tests,https://en.yna.co.kr/view/AEN20221009000354325...,2022,10,Saturday
201,2022-10-08,2022-10-10,16:47:00,KN-25,SRBM,Munchon,"Munchon, Kangwong Province",39.2815,127.377,Sea of Japan or East Sea,90 km,350 km,Confirmed,Success,"On October 9 (local time), North Korea launche...",https://en.yna.co.kr/view/AEN20221009000354325...,2022,10,Saturday
200,2022-10-05,2022-10-06,21:15:00,KN-23,SRBM,Samsok,"Samsok District, Pyongyang",39.1268,125.9579,Sea of Japan or East Sea,50 km,800 km,Confirmed,Success,2 of 2 tests,https://en.yna.co.kr/view/AEN20221006000853325...,2022,10,Wednesday
199,2022-10-05,2022-10-06,21:00:00,KN-25,SRBM,Samsok,"Samsok District, Pyongyang",39.1268,125.9579,Sea of Japan or East Sea,100 km,350 km,Confirmed,Success,"Less than 24 hours after the IRBM launch, Nort...",https://en.yna.co.kr/view/AEN20221006000853325...,2022,10,Wednesday
198,2022-10-03,2022-10-06,22:22:00,New IRBM (2022),IRBM,Mupyong-ni Arms Plant,"Mup'yong-ni, Chagang province",40.611208,126.425743,Pacific Ocean,970 km,4500 km,Confirmed,Success,"At 7:22 AM local time on 4 October 2022, North...",https://en.yna.co.kr/view/AEN20221004001354325...,2022,10,Monday


---

In [95]:
missile_types = (
    df.groupby(["year", "missile_type"])["date"].count().reset_index(name="count")
)

In [96]:
missile_types

,year,missile_type,count
0,1984,SRBM,6
1,1986,Unknown,1
2,1990,MRBM,1
3,1990,SRBM,1
4,1991,SRBM,1
5,1992,MRBM,1
6,1993,MRBM,1
7,1993,SRBM,3
8,1998,SLV,1
9,2006,MRBM,4


---

In [113]:
alt.Chart(missile_types).mark_circle(
    opacity=0.7, stroke="#1a1a1a", strokeWidth=0.5
).encode(
    alt.X(
        "year:O",
        axis=alt.Axis(tickCount=2, values=["1984", "2022"]),
        title="",
    ),
    alt.Y("missile_type:N", title=""),
    alt.Size(
        "count:Q",
        scale=alt.Scale(range=[0, 5000]),
        legend=None,
    ),
    alt.Color("missile_type:N", legend=None),
).properties(
    width=360, height=500, title="North Korea missile launches by type and year"
).configure_legend(
    orient="top", symbolType="circle"
)

alt.Chart(...)